# Application Overview
-  Patients click a button to start typing basic user information
-  Gradio interface gather user informations and pass into Supervised-tuned Vertax AI
-  The application detect patients has finished typing and entering
-  The patients messages passed to the Vertax AI model, which streams back a response
-  After Supervised Vertax AI finishes typing the patient message bar is reactiviated
-  All previous messages from both the user and Vertax AI, is displayed in a
chatbot component

## Install the Python packages

In [10]:
! pip install --upgrade gradio

In [11]:
pip install google-generativeai google-cloud-storage

In [12]:
!pip install --upgrade google-cloud-aiplatform vertexai

  Using cached google_cloud_aiplatform-1.89.0-py2.py3-none-any.whl.metadata (35 kB)


## Load and Prepare the Counseling Dataset

In [13]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('20200325_counsel_chat.csv')

# Select and rename the columns
selected_column = df[['questionText', 'answerText']]
selected_column.rename(columns={'questionText': 'role', 'answerText': 'content'}, inplace=True)

<ipython-input-13-7fd48a5315ef>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_column.rename(columns={'questionText': 'role', 'answerText': 'content'}, inplace=True)


## Remove Duplicate Questions

In [14]:
# Drop duplicate questions
selected_column.drop_duplicates(subset='role', inplace=True)

<ipython-input-14-f555d65cd955>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_column.drop_duplicates(subset='role', inplace=True)


## Convert CSV Data to JSONL Format for Fine-Tuning

In [15]:
import json

# Convert DataFrame to the JSON format
formatted_data = selected_column.apply(
    lambda row: {
        "systemInstruction": {
            "role": "system",
            "parts": [{"text": "Provide counseling responses"}]
        },
        "contents": [
            {
                "role": "user",
                "parts": [{"text": row['role']}]
            },
            {
                "role": "model",
                "parts": [{"text": row['content']}]
            }
        ]
    }, axis=1).tolist()

# Save the formatted data to a JSONL file
with open("counsel_chat.jsonl", "w") as jsonl_output:
    for entry in formatted_data:
        json.dump(entry, jsonl_output)
        jsonl_output.write("\n")

# Print first 5 lines of the JSONL file
with open("counsel_chat.jsonl", "r") as f:
    for _ in range(5):
        print(json.loads(f.readline()))

{'systemInstruction': {'role': 'system', 'parts': [{'text': 'Provide counseling responses'}]}, 'contents': [{'role': 'user', 'parts': [{'text': "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?"}]}, {'role': 'model', 'parts': [{'text': "If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media. \xa0Maybe read some of the ones which state that no person is worthless, and that everyone

### Upload above JSONL file to the Vetax AI and supervised-tuned so it can learn the reponse from real counsler

# Connect to Vertex AI and Load the Fine-Tuned Model


In [16]:
from google.cloud import aiplatform
from vertexai.generative_models import GenerativeModel

# Initialize Vertex AI
aiplatform.init(project="1084816407090", location="us-central1")

# List all models
models = aiplatform.Model.list()
for model in models:
    print(model.name, model.display_name)

# Load your fine-tuned AI Therapist2 model
model_name = "projects/1084816407090/locations/us-central1/models/279818012687597568"
tuned_model = GenerativeModel(model_name=model_name)

279818012687597568 AI Therapist2
4402669957498798080 tuned-gemini-pro-20250308-214215
7087941235318456320 tuned-gemini-pro-20250308-214215
4459527902794350592 tuned-gemini-pro-20250308-214215


# Deploy the Fine-Tuned Model to an Endpoint

In [17]:
model = aiplatform.Model("projects/1084816407090/locations/us-central1/models/279818012687597568")

# create new endpoint everytime it runs
endpoint = model.deploy(
    machine_type="n1-standard-4",  # Change if needed
    min_replica_count=1,
    traffic_split={"0": 100},
)
print("Model deployed at:", endpoint.resource_name)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/1084816407090/locations/us-central1/endpoints/127109041729044480/operations/2808975784872509440
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/1084816407090/locations/us-central1/endpoints/127109041729044480
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/1084816407090/locations/us-central1/endpoints/127109041729044480')
INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/1084816407090/locations/us-central1/endpoints/127109041729044480
INFO:google.cloud.aiplatform.models:Model does not support dedicated deployment resources. The machine_type, accelerator_type and accelerator_count,autoscaling_target_accelerator_duty_cycle,autoscaling_target_cpu_utilization parameters are ignored.
INFO:google.cloud.aipla

Model deployed at: projects/1084816407090/locations/us-central1/endpoints/127109041729044480


# Build an Interactive AI Therapist UI with Gradio

In [18]:
import gradio as gr
from google.cloud import aiplatform
from vertexai.generative_models import GenerativeModel

# Initialize Vertex AI
aiplatform.init(project="1084816407090", location="us-central1")

# Load your fine-tuned AI Therapist model
tuned_model = GenerativeModel(model_name="projects/1084816407090/locations/us-central1/endpoints/8428386647065755648")

# Store user profile and conversation history
user_profile = {}
conversation_history = []

# Function to collect user details
def collect_user_info(name, age, concerns, mental_health_history):
    global user_profile
    if not name or not age or not concerns:
        return "I'm sorry, but it seems like you haven't filled out all the required information. Please provide your name, age, and main concerns. 🙏"

    user_profile = {
        "name": name,
        "age": age,
        "concerns": concerns,
        "mental_health_history": mental_health_history
    }
    return f"Thank you, {name}. Your information has been saved. You may now start counseling. 🐶😊"

# Function to interact with the AI Therapist model
def chat_with_model(user_input):
    if not user_profile or not user_profile.get('name') or not user_profile.get('age') or not user_profile.get('concerns'):
        return "I'm sorry, but it seems like we don't have enough information to proceed. Please provide your name, age, and main concerns. 🙏"

    # Update conversation history
    conversation_history.append(f"User: {user_input}")

    # Constructing personalized prompt with conversation history
    prompt = (
        "You are an AI therapist named AI Therapist. Consider the user's background while responding as a counselor.\n Do not create advice such as seek out counseling with a licensed therapist name. You should engage with the user in a supportive and empathetic manner, offering words of encouragement and motivation.\n Your primary goal is to create a safe and non-judgmental space for the user to express their thoughts and feelings.\n You should use gentle prompts and positive reinforcement to encourage the user to open up and share their inner thoughts and emotions.\n Ensure that you maintain a respectful and empathetic tone throughout the conversation.\n Do not include any real counselor's name in your response.\n"
        "User Profile:\n"
        f"- Name: {user_profile['name']}\n"
        f"- Age: {user_profile['age']}\n"
        f"- Concerns: {user_profile['concerns']}\n"
        f"- Mental Health History: {user_profile['mental_health_history']}\n\n"
        "Conversation History:\n"
        f"{''.join(conversation_history)}\n\n"
        "AI Therapist2 (as a counselor):"
    )

    # Generate response using the fine-tuned model
    response = tuned_model.generate_content(prompt)
    conversation_history.append(f"AI Therapist: {response.text}")
    return response.text, "\n".join(conversation_history)

# Gradio UI
with gr.Blocks(theme='hmb/amethyst') as app:
    gr.Markdown("## 🐶 AI Therapist - Healing Space 😊")
    with gr.Tab("Step 1: Enter Your Information"):
        name = gr.Textbox(label="Your Name")
        age = gr.Textbox(label="Your Age")
        concerns = gr.Textbox(label="Your Main Concerns")
        mental_health_history = gr.Textbox(label="Any Mental Health History (Optional)")
        submit_button = gr.Button("Save Information")
        output_text = gr.Textbox(label="Confirmation", interactive=False)

        submit_button.click(collect_user_info, inputs=[name, age, concerns, mental_health_history], outputs=output_text)

    with gr.Tab("Step 2: Start Counseling"):
        chat_input = gr.Textbox(label="Ask your therapist")
        chat_button = gr.Button("Ask")
        chat_output = gr.Textbox(label="Therapist's Response", interactive=False)
        chat_history = gr.Textbox(label="Conversation History", interactive=False)

        chat_button.click(chat_with_model, inputs=chat_input, outputs=[chat_output, chat_history])

# Launch the Gradio app
app.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5ebc36fa0a81b122e8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
